# Handling GSM8K



In [18]:
output_file_path = "/data/home/chensh/projects/LLM_router/datasets/split2_model7"

In [17]:
model_list = [  
                ['mistralai','Mistral-7B-v0.1'],
                ['meta-math','MetaMath-Mistral-7B'],
                ['itpossible','Chinese-Mistral-7B-v0.1'],
                ["HuggingFaceH4","zephyr-7b-beta"],
                ["cognitivecomputations","dolphin-2.6-mistral-7b"],
                ["meta-llama","Meta-Llama-3-8B"],
                ["cognitivecomputations","dolphin-2.9-llama3-8b"],
                ]

In [19]:

# /data/home/chensh/projects/LLM_router/output/humaneval/Arithmo2-Mistral-7B_humaneval.json

# for mbpp

import os
import json
import pandas as pd
import random
random.seed(42)

task = "mbpp"

mbpp_data = pd.read_parquet("/data/home/chensh/data/LLM_datasets/mbpp/full/test-00000-of-00001.parquet",engine='pyarrow')

output_data = []
for model_index, model_info in enumerate(model_list):
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/mbpp/{model}/log_samples.json",'r') as f:
        json_file = json.load(f)
    json_file = json_file['mbpp']
    for index, instance in json_file.items():
        correct = 0
        for each_resps in instance:
            correct += 1 if each_resps[1]['passed'] == True else 0
        acc = correct / len(instance)
        description = mbpp_data.iloc[int(index)]["text"]
        test_example = mbpp_data.iloc[int(index)]["test_list"][0]
        question = f'"""\n{description}\n{test_example}\n"""\n'
        if model_index == 0:
            output_data.append({"question": question, "scores": {model_pre + "/" + model: acc}})
        else:
            output_data[int(index)]["scores"][model_pre+"/" + model] = acc

print(len(output_data))
# train_split_index = random.sample(range(len(output_data)), len(output_data))
# output_data = [output_data[index] for index in train_split_index]
# train_split = output_data[:115]
# test_split = output_data[115:]
# test_split = output_data[100:]

with open(os.path.join(output_file_path, 'mbpp.json'), 'w') as f:
    json.dump(output_data, f)
#     
# train_split[0]


500


In [25]:

# /data/home/chensh/projects/LLM_router/output/humaneval/Arithmo2-Mistral-7B_humaneval.json

# for javascript
model_list = [  
                ['mistralai','Mistral-7B-v0.1'],
                ['meta-math','MetaMath-Mistral-7B'],
                ['itpossible','Chinese-Mistral-7B-v0.1'],
                ["HuggingFaceH4","zephyr-7b-beta"],
                ["cognitivecomputations","dolphin-2.6-mistral-7b"],
                ["meta-llama","Meta-Llama-3-8B"],
                ["cognitivecomputations","dolphin-2.9-llama3-8b"],
                ]

output_file_path = "/data/home/chensh/projects/LLM_router/datasets/split2_model7"

import os
import json
import pandas as pd
import random
random.seed(42)

task = "javascript"

javascript_data = pd.read_json("/data/home/chensh/data/LLM_datasets/bigcode/humanevalpack/data/js/data/humanevalpack.jsonl", lines=True)

print(javascript_data.iloc[0]['prompt'])

/* Check if in given list of numbers, are any two numbers closer to each other than
  given threshold.
  >>> hasCloseElements([1.0, 2.0, 3.0], 0.5)
  false
  >>> hasCloseElements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
  true
  */
const hasCloseElements = (numbers, threshold) => {



In [26]:

output_data = []
for model_index, model_info in enumerate(model_list):
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/humanevalpack_js/{model}/log_samples.json",'r') as f:
        json_file = json.load(f)
    json_file = json_file['humanevalsynthesize-js']
    for index, instance in json_file.items():
        correct = 0
        for each_resps in instance:
            correct += 1 if each_resps[1]['passed'] == True else 0
        acc = correct / len(instance)
        question = javascript_data.iloc[int(index)]["prompt"][0]
        if model_index == 0:
            output_data.append({"question": question, "scores": {model_pre + "/" + model: acc}})
        else:
            output_data[int(index)]["scores"][model_pre + "/" + model] = acc

print(len(output_data))
# train_split_index = random.sample(range(len(output_data)), len(output_data))
# output_data = [output_data[index] for index in train_split_index]
# train_split = output_data[:115]
# test_split = output_data[115:]
# test_split = output_data[100:]

with open(os.path.join(output_file_path, 'javascript.json'), 'w') as f:
    json.dump(output_data, f)
#     
# train_split[0]


164


In [9]:

import json
import os
import re
import time
import numpy as np

# for gsm8k

task = "gsm8k_train"
# task = "gsm8k-repeat10"

output_data = []
for model_index, model_info in enumerate(model_list):
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/gsm8k_train-t0.2/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_{task}.jsonl", 'r') as f:
        json_file = json.load(f)
    for i in range(len(json_file)):
        # extract answer_list & answer
        resps = json_file[i]['resps'][0]
        pattern = "#### (\\-?[0-9\\.\\,]+)"
        ans_list = [re.search(pattern, resp).group(1) if re.search(pattern, resp) else None for resp in resps]
        target = re.search(pattern, json_file[i]['target']).group(1)
         # compare with origin answer & get score
        correct = 0
        for ans in ans_list:
            correct += 1 if ans == target else 0 
        acc = correct / len(ans_list)
        if model_index == 0:
            output_data.append({"question":json_file[i]['doc']["question"], "scores": {model_pre+"/" + model: acc}})
        else:
            output_data[i]["scores"][model_pre+"/" + model] = acc
            
output_data = output_data[:int(len(output_data)/2)]
with open(os.path.join(output_file_path, 'gsm8k-train.json'), 'w') as f:
    json.dump(output_data, f)


In [13]:
# modified_model_names = ["Meta-Llama-3-8B","Llama3-8B-Chinese-Chat","dolphin-2.9-llama3-8b"]
# for modified_model_name in modified_model_names:
#     path = os.path.join("/data/home/chensh/projects/LLM_router/output/mmlu_5shot", modified_model_name)
#     files = os.listdir(path)
#     for file in files:
#         print(file)
#         if file.startswith("pretrained"):
#             os.rename(os.path.join(path, file), os.path.join(path, file.replace(",max_model_len__4096", "")))        

# MMLU

In [10]:
import json
import os
import time
import numpy as np
import random
random.seed(42)

# for mmlu (multiple choice problem)

task_list = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering', 'elementary_mathematics', 'formal_logic', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science', 'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 'high_school_macroeconomics', 'high_school_mathematics', 'high_school_microeconomics', 'high_school_physics', 'high_school_psychology', 'high_school_statistics', 'high_school_us_history', 'high_school_world_history', 'human_aging', 'human_sexuality', 'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 'management', 'marketing', 'medical_genetics', 'miscellaneous', 'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    choices_head = ['A. ', 'B. ', 'C. ', 'D. ']
    for task in task_list:
        model_pre = model_info[0]
        model = model_info[1]
        with open(f"/data/home/chensh/projects/LLM_router/output/mmlu_5shot/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_mmlu_{task}.jsonl", 'r') as f:
            json_file = json.load(f)
        for i in range(len(json_file)):
            resps = np.array(json_file[i]['resps'])
            log_probability = resps[:,0,0]
            probability = np.exp(log_probability)
            sum_probability = np.sum(probability)
            probability = probability / sum_probability
            score = probability[json_file[i]['target']] if json_file[i]['acc'] == 1 else 0

            if model_index == 0:
                output_data.append(
                    {"question":json_file[i]['doc']["question"] + 
                                "".join(["\n" + choices_head[j] + choice for j, choice in enumerate(json_file[i]['doc']['choices'])]), "scores": {model_pre+"/" + model: score}}
                )
            else:
                output_data[count]["scores"][model_pre+"/" + model] = score
                count += 1

train_split_index = random.sample(range(len(output_data)), len(output_data))
output_data = [output_data[index] for index in train_split_index]
train_split = output_data[:int(0.7* len(output_data))]
test_split = output_data[int(0.7* len(output_data)):]

# print(len(output_data))
# output training data as a json file
with open(os.path.join(output_file_path, f'mmlu_train.json'), 'w') as f:
    json.dump(train_split, f)

with open(os.path.join(output_file_path, f'mmlu_test.json'), 'w') as f:
    json.dump(test_split, f) 


# Humaneval

In [11]:

# /data/home/chensh/projects/LLM_router/output/humaneval/Arithmo2-Mistral-7B_humaneval.json

# for humaneval

import os
import json
import pandas as pd
import random
random.seed(42)

task = "humaneval"

humaneval_data = pd.read_parquet("/data/home/chensh/data/LLM_datasets/openai_humaneval/openai_humaneval/test-00000-of-00001.parquet",engine='pyarrow')

output_data = []
for model_index, model_info in enumerate(model_list):
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/humaneval/{model}/log_samples.json",'r') as f:
        json_file = json.load(f)
    json_file = json_file['humaneval']
    for index, instance in json_file.items():
        correct = 0
        for each_resps in instance:
            correct += 1 if each_resps[1]['passed'] == True else 0
        acc = correct / len(instance)
        question = humaneval_data.iloc[int(index)]['prompt']
        if model_index == 0:
            output_data.append({"question": question, "scores": {model_pre + "/" + model: acc}})
        else:
            output_data[int(index)]["scores"][model_pre+"/" + model] = acc

train_split_index = random.sample(range(len(output_data)), len(output_data))
output_data = [output_data[index] for index in train_split_index]
train_split = output_data[:115]
test_split = output_data[115:]
# test_split = output_data[100:]

with open(os.path.join(output_file_path, 'humaneval_train.json'), 'w') as f:
    json.dump(train_split, f)

with open(os.path.join(output_file_path, 'humaneval_test.json'), 'w') as f:
    json.dump(test_split, f) 

print(len(train_split))
print(len(test_split))
#     
# train_split[0]


115
49


# ARC

In [12]:
import json
import os
import time
import numpy as np
import random
random.seed(100)

# for arc (multiple choice problem)

# task = "arc_easy"
task = "arc_challenge"

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/{task}/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_{task}.jsonl", 'r') as f:
        json_file = json.load(f)
    for i in range(len(json_file)):
        resps = np.array(json_file[i]['resps'])
        log_probability = resps[:,0,0]
        probability = np.exp(log_probability)
        sum_probability = np.sum(probability)
        probability = probability / sum_probability
        score = probability[json_file[i]['target']] if json_file[i]['acc'] == 1 else 0

        if model_index == 0:
            output_data.append(
                {"question": "Question:" + json_file[i]['doc']["question"] + "\nAnswer:", "scores": {model_pre+"/" + model: score}}
            )
        else:
            output_data[count]["scores"][model_pre + "/" + model] = score
            count += 1

train_split_index = random.sample(range(len(output_data)), len(output_data))
output_data = [output_data[index] for index in train_split_index]
train_split = output_data[:int(0.7* len(output_data))]
test_split = output_data[int(0.7* len(output_data)):]
       

# output training data as a json file
# output_file_path = "/data/home/chensh/projects/LLM_router/datasets/"
# with open(os.path.join(output_file_path, f'{task}.json'), 'w') as f:
#     json.dump(output_data, f)

with open(os.path.join(output_file_path, f'{task}_train.json'), 'w') as f:
    json.dump(train_split, f)

with open(os.path.join(output_file_path, f'{task}_test.json'), 'w') as f:
    json.dump(test_split, f) 



# Hellaswag

In [12]:
import json
import os
import time
import numpy as np
import re

# for hellaswag (multiple choice problem)

task = "hellaswag"

def preprocess(text):
    text = text.strip()
    # NOTE: Brackets are artifacts of the WikiHow dataset portion of HellaSwag.
    text = text.replace(" [title]", ". ")
    text = re.sub("\\[.*?\\]", "", text)
    text = text.replace("  ", " ")
    return text

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/{task}_train/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_{task}.jsonl", 'r') as f:
        json_file = json.load(f)
    for i in range(len(json_file)):
        resps = np.array(json_file[i]['resps'])
        log_probability = resps[:,0,0]
        probability = np.exp(log_probability)
        sum_probability = np.sum(probability)
        probability = probability / sum_probability
        score = probability[json_file[i]['target']] if json_file[i]['acc_norm'] == 1 else 0
        question = json_file[i]['doc']["ctx_a"] + " " + json_file[i]['doc']["ctx_b"].capitalize()
        question = preprocess(json_file[i]['doc']["activity_label"] + ": " + question)

        if model_index == 0:
            output_data.append(
                {"question": question, "scores": {model_pre+"/" + model: score}}
            )
        else:
            output_data[count]["scores"][model_pre + "/" + model] = score
            count += 1
       
# output training data as a json file
# output_data = output_data[:1000]
with open(os.path.join(output_file_path, f'hellaswag_validation.json'), 'w') as f:
    json.dump(output_data, f)

# len(output_data)

output_data[0]


{'question': 'Removing ice from car: Then, the man writes over the snow covering the window of a car, and a woman wearing winter clothes smiles. Then',
 'scores': {'mistralai/Mistral-7B-v0.1': 0.9999965516457299,
  'meta-math/MetaMath-Mistral-7B': 0.9997326286037237,
  'teknium/OpenHermes-2.5-Mistral-7B': 0.9999999632149246,
  'WizardLM/WizardMath-7B-V1.1': 0.9999973056897147,
  'itpossible/Chinese-Mistral-7B-v0.1': 0.9999980523735081,
  'HuggingFaceH4/zephyr-7b-beta': 0.9999996688090383,
  'cognitivecomputations/dolphin-2.6-mistral-7b': 0.9999998706397251,
  'NousResearch/Hermes-2-Pro-Mistral-7B': 0.9999999561210721,
  'cognitivecomputations/dolphin-2.2.1-mistral-7b': 0.9999994994896325,
  'abhishekchohan/mistral-7B-forest-dpo': 0.9999999418205058}}

# C-Eval

In [13]:
import json
import os
import time
import numpy as np
import random


# modified_model_names = ["Meta-Llama-3-8B","Llama3-8B-Chinese-Chat","dolphin-2.9-llama3-8b"]
# for modified_model_name in modified_model_names:
#     path = os.path.join("/data/home/chensh/projects/LLM_router/output/ceval-validation", modified_model_name)
#     files = os.listdir(path)
#     for file in files:
#         if file.startswith("pretrained"):
#             os.rename(os.path.join(path, file), os.path.join(path, file.replace(",max_model_len__4096", "")))        

# for ceval (multiple choice problem)

task_list = ['computer_network', 'operating_system', 'computer_architecture', 'college_programming', 'college_physics', 'college_chemistry', 'advanced_mathematics', 'probability_and_statistics', 'discrete_mathematics', 'electrical_engineer', 'metrology_engineer', 'high_school_mathematics', 'high_school_physics', 'high_school_chemistry', 'high_school_biology', 'middle_school_mathematics', 'middle_school_biology', 'middle_school_physics', 'middle_school_chemistry', 'veterinary_medicine', 'college_economics', 'business_administration', 'marxism', 'mao_zedong_thought', 'education_science', 'teacher_qualification', 'high_school_politics', 'high_school_geography', 'middle_school_politics', 'middle_school_geography', 'modern_chinese_history', 'ideological_and_moral_cultivation', 'logic', 'law', 'chinese_language_and_literature', 'art_studies', 'professional_tour_guide', 'legal_professional', 'high_school_chinese', 'high_school_history', 'middle_school_history', 'civil_servant', 'sports_science', 'plant_protection', 'basic_medicine', 'clinical_medicine', 'urban_and_rural_planner', 'accountant', 'fire_engineer', 'environmental_impact_assessment_engineer', 'tax_accountant', 'physician']

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    choices_head = ['A', 'B', 'C', 'D']
    for task in task_list:
        model_pre = model_info[0]
        model = model_info[1]
        with open(f"/data/home/chensh/projects/LLM_router/output/ceval-validation/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_ceval-valid_{task}.jsonl", 'r') as f:
            json_file = json.load(f)
        for i in range(len(json_file)):
            resps = np.array(json_file[i]['resps'])
            log_probability = resps[:,0,0]
            probability = np.exp(log_probability)
            sum_probability = np.sum(probability)
            probability = probability / sum_probability
            score = probability[json_file[i]['target']] if json_file[i]['acc'] == 1 else 0

            if model_index == 0:
                output_data.append(
                    {"question":json_file[i]['doc']["question"] + 
                                "".join(["\n" + choices_head[j] + ". " + json_file[i]['doc'][choices_head[j]] for j in range(4)]) + "答案：", "scores": {model_pre+"/" + model: score}}
                )
            else:
                output_data[count]["scores"][model_pre+"/" + model] = score
                count += 1


# output training data as a json file
# train_split_index = random.sample(range(len(output_data)), len(output_data))
# output_data = [output_data[index] for index in train_split_index]
# train_split = output_data[:1000]
# test_split = output_data[1000:]


# output training data as a json file
with open(os.path.join(output_file_path, 'ceval.json'), 'w') as f:
    json.dump(output_data, f)

# len(output_data)

# CMMLU

In [14]:
import json
import os
import time
import numpy as np
import random
random.seed(52)

# modified_model_names = ["Meta-Llama-3-8B","Llama3-8B-Chinese-Chat","dolphin-2.9-llama3-8b"]
# for modified_model_name in modified_model_names:
#     path = os.path.join("/data/home/chensh/projects/LLM_router/output/cmmlu", modified_model_name)
#     files = os.listdir(path)
#     for file in files:
#         if file.startswith("pretrained"):
#             os.rename(os.path.join(path, file), os.path.join(path, file.replace(",max_model_len__4096", "")))    

# for cmmlu (multiple choice problem)

task_list = ['agronomy', 'anatomy', 'ancient_chinese', 'arts', 'astronomy', 'business_ethics', 'chinese_civil_service_exam', 'chinese_driving_rule', 'chinese_food_culture', 'chinese_foreign_policy', 'chinese_history', 'chinese_literature', 'chinese_teacher_qualification', 'clinical_knowledge', 'college_actuarial_science', 'college_education', 'college_engineering_hydrology', 'college_law', 'college_mathematics', 'college_medical_statistics', 'college_medicine', 'computer_science', 'computer_security', 'conceptual_physics', 'construction_project_management', 'economics', 'education', 'electrical_engineering', 'elementary_chinese', 'elementary_commonsense', 'elementary_information_and_technology', 'elementary_mathematics', 'ethnology', 'food_science', 'genetics', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_geography', 'high_school_mathematics', 'high_school_physics', 'high_school_politics', 'human_sexuality', 'international_law', 'journalism', 'jurisprudence', 'legal_and_moral_basis', 'logical', 'machine_learning', 'management', 'marketing', 'marxist_theory', 'modern_chinese', 'nutrition', 'philosophy', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_study', 'sociology', 'sports_science', 'traditional_chinese_medicine', 'virology', 'world_history', 'world_religions']

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    choices_head = ['A', 'B', 'C', 'D']
    for task in task_list:
        model_pre = model_info[0]
        model = model_info[1]
        with open(f"/data/home/chensh/projects/LLM_router/output/cmmlu/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_cmmlu_{task}.jsonl", 'r') as f:
            json_file = json.load(f)
        for i in range(len(json_file)):
            resps = np.array(json_file[i]['resps'])
            log_probability = resps[:,0,0]
            probability = np.exp(log_probability)
            sum_probability = np.sum(probability)
            probability = probability / sum_probability
            score = probability[json_file[i]['target']] if json_file[i]['acc'] == 1 else 0

            if model_index == 0:
                output_data.append(
                    {"question":json_file[i]['doc']['Question'].strip() + "".join(["\n" + choices_head[j] + ". " + json_file[i]['doc'][choices_head[j]] for j in range(4)]) + "\n答案：", "scores": {model_pre+"/" + model: score}}
                )
            else:
                output_data[count]["scores"][model_pre+"/" + model] = score
                count += 1
                
train_split_index = random.sample(range(len(output_data)), len(output_data))
output_data = [output_data[index] for index in train_split_index]
train_split = output_data[:int(0.7* len(output_data))]
test_split = output_data[int(0.7* len(output_data)):]


# output training data as a json file
with open(os.path.join(output_file_path, 'cmmlu_train.json'), 'w') as f:
    json.dump(train_split, f)

with open(os.path.join(output_file_path, 'cmmlu_test.json'), 'w') as f:
    json.dump(test_split, f) 
# json_file[i]['arguments']

# MATH

In [15]:
import json
import os
import time
import numpy as np
import random
random.seed(52)


# for cmmlu (multiple choice problem)

# task_list = ["minerva_math_precalc", "minerva_math_prealgebra", "minerva_math_num_theory", "minerva_math_intermediate_algebra", "minerva_math_geometry", "minerva_math_counting_and_prob", "minerva_math_algebra"]

task_list = ["minerva_math_prealgebra"]

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    for task in task_list:
        model_pre = model_info[0]
        model = model_info[1]
        with open(f"/data/home/chensh/projects/LLM_router/output/MATH/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_{task}.jsonl", 'r') as f:
            json_file = json.load(f)
        for i in range(len(json_file)):
            if model_index == 0:
                score = json_file[i]['exact_match']
                output_data.append(
                    {"question": json_file[i]['doc']['problem'], "scores": {model_pre+"/" + model: score}}
                )
            else:
                score = json_file[i]['exact_match']
                output_data[count]["scores"][model_pre+"/" + model] = score
                count += 1


# output training data as a json file
with open(os.path.join(output_file_path, 'MATH_prealgebra.json'), 'w') as f:
    json.dump(output_data, f)

In [2]:
import numpy as np
import pandas as pd
import os
import json

output_file_path = "/data/home/chensh/projects/LLM_router/datasets/split2_model8"
with open(os.path.join(output_file_path, 'arc_challenge_train.json'),'r') as f: 
    output_data = json.load(f)


count_dict = {
'mistralai/Mistral-7B-v0.1': 0,
'meta-math/MetaMath-Mistral-7B' :0, 
'teknium/OpenHermes-2.5-Mistral-7B':0,
# 'upaya07/Arithmo2-Mistral-7B':0,
# 'ajibawa-2023/Code-Mistral-7B':0,
'WizardLM/WizardMath-7B-V1.1':0,
'itpossible/Chinese-Mistral-7B-v0.1':0,
'HuggingFaceH4/zephyr-7b-beta':0,
'cognitivecomputations/dolphin-2.6-mistral-7b':0,
# 'Open-Orca/Mistral-7B-OpenOrca':0,
# 'HuggingFaceH4/mistral-7b-grok':0,
# 'HuggingFaceH4/mistral-7b-sft-beta':0,
'NousResearch/Hermes-2-Pro-Mistral-7B':0,
'cognitivecomputations/dolphin-2.2.1-mistral-7b':0,
'abhishekchohan/mistral-7B-forest-dpo':0,
# 'wandb/mistral-7b-zephyr-sft':0,
# 'Locutusque/Hercules-2.5-Mistral-7B':0,
# 'nvidia/OpenMath-Mistral-7B-v0.1-hf':0,
"meta-llama/Meta-Llama-3-8B":0,
"shenzhi-wang/Llama3-8B-Chinese-Chat":0,
"cognitivecomputations/dolphin-2.9-llama3-8b":0,
}

print(len(output_data))

# print(output_data[i])
for i in range(len(output_data)):
    sorted_dict = dict(sorted(output_data[i]['scores'].items(), key=lambda x: x[1], reverse=True))
    # max_value = max(output_data[i]['scores'].values())
    # it = iter([index for index, value in output_data[i]['scores'].items() if value == max_value])
    for index, (key, value) in enumerate(sorted_dict.items()):
        if index > 1:
            break
        if value > 0:
            count_dict[key] += 1
    
print(sum(count_dict.values()))

count_dict

820
1138


{'mistralai/Mistral-7B-v0.1': 29,
 'meta-math/MetaMath-Mistral-7B': 109,
 'teknium/OpenHermes-2.5-Mistral-7B': 0,
 'WizardLM/WizardMath-7B-V1.1': 0,
 'itpossible/Chinese-Mistral-7B-v0.1': 35,
 'HuggingFaceH4/zephyr-7b-beta': 308,
 'cognitivecomputations/dolphin-2.6-mistral-7b': 115,
 'NousResearch/Hermes-2-Pro-Mistral-7B': 0,
 'cognitivecomputations/dolphin-2.2.1-mistral-7b': 0,
 'abhishekchohan/mistral-7B-forest-dpo': 0,
 'meta-llama/Meta-Llama-3-8B': 70,
 'shenzhi-wang/Llama3-8B-Chinese-Chat': 243,
 'cognitivecomputations/dolphin-2.9-llama3-8b': 229}



# Get ACC

In [27]:
import pandas as pd
import os
import json

output_file_path = "/data/home/chensh/projects/LLM_router/datasets/split2_model7"
with open(os.path.join(output_file_path, 'javascript.json'),'r') as f: 
    output_data = json.load(f)

correct_dict = {
'mistralai/Mistral-7B-v0.1': 0,
'meta-math/MetaMath-Mistral-7B' :0, 
'teknium/OpenHermes-2.5-Mistral-7B':0,
# 'upaya07/Arithmo2-Mistral-7B':0,
# 'ajibawa-2023/Code-Mistral-7B':0,
'WizardLM/WizardMath-7B-V1.1':0,
'itpossible/Chinese-Mistral-7B-v0.1':0,
'HuggingFaceH4/zephyr-7b-beta':0,
'cognitivecomputations/dolphin-2.6-mistral-7b':0,
# 'Open-Orca/Mistral-7B-OpenOrca':0,
# 'HuggingFaceH4/mistral-7b-grok':0,
# 'HuggingFaceH4/mistral-7b-sft-beta':0,
'NousResearch/Hermes-2-Pro-Mistral-7B':0,
'cognitivecomputations/dolphin-2.2.1-mistral-7b':0,
'abhishekchohan/mistral-7B-forest-dpo':0,
"meta-llama/Meta-Llama-3-8B":0,
"shenzhi-wang/Llama3-8B-Chinese-Chat":0,
"cognitivecomputations/dolphin-2.9-llama3-8b":0,
"meta-llama/Llama-2-13b-hf": 0,
# 'wandb/mistral-7b-zephyr-sft':0,
# 'Locutusque/Hercules-2.5-Mistral-7B':0,
# 'nvidia/OpenMath-Mistral-7B-v0.1-hf':0,
}

# print(output_data[0])

data_size = len(output_data)
print(data_size)

for item in output_data:
    scores = item['scores']
    for key, score in scores.items():
        # if score > 0: 
        correct_dict[key] += score / data_size * 100

dataframe = pd.DataFrame.from_dict(correct_dict, orient='index')
dataframe
# for key, value in correct_dict.items():
#     print(key,value)
# print(correct_dict)

164


,0
mistralai/Mistral-7B-v0.1,29.878049
meta-math/MetaMath-Mistral-7B,31.829268
teknium/OpenHermes-2.5-Mistral-7B,0.000000
WizardLM/WizardMath-7B-V1.1,0.000000
itpossible/Chinese-Mistral-7B-v0.1,17.682927
HuggingFaceH4/zephyr-7b-beta,11.707317
cognitivecomputations/dolphin-2.6-mistral-7b,45.000000
NousResearch/Hermes-2-Pro-Mistral-7B,0.000000
cognitivecomputations/dolphin-2.2.1-mistral-7b,0.000000
abhishekchohan/mistral-7B-forest-dpo,0.000000


# Split for fewer model

In [1]:
import os
import json
model_list = [  
                ['mistralai','Mistral-7B-v0.1'],
                ['meta-math','MetaMath-Mistral-7B'],
                ['itpossible','Chinese-Mistral-7B-v0.1'],
                ["HuggingFaceH4","zephyr-7b-beta"],
                ["cognitivecomputations","dolphin-2.6-mistral-7b"],
                ["meta-llama","Meta-Llama-3-8B"],
                # ["shenzhi-wang","Llama3-8B-Chinese-Chat"],
                ["cognitivecomputations","dolphin-2.9-llama3-8b"],
                ]
model_list = [item[0] + '/' + item[1] for item in model_list]


source_output_path = "/data/home/chensh/projects/LLM_router/datasets/split2_model8_2"
target_output_path = "/data/home/chensh/projects/LLM_router/datasets/split2_model8_zero_test"
os.makedirs(target_output_path)


source_dataset_list = os.listdir(source_output_path)
for source_dataset in source_dataset_list:
    with open(os.path.join(source_output_path, source_dataset),'r') as f: 
        source_data = json.load(f)
    target_data = []
    for item in source_data:
        target_item = item
        target_score = {}
        for key, value in item['scores'].items():
            if key == "meta-llama/Llama-2-13b-hf":
                target_score[key] = 0
            if key in model_list:
                target_score[key] = value
        target_item['scores'] = target_score
        target_data.append(target_item)

    with open(os.path.join(target_output_path, source_dataset),'w') as f: 
        json.dump(target_data, f)



In [9]:
json_file[0]

{'doc_id': 0,
 'doc': {'ind': 24,
  'activity_label': 'Roof shingle removal',
  'ctx_a': 'A man is sitting on a roof.',
  'ctx_b': 'he',
  'ctx': 'A man is sitting on a roof. he',
  'endings': ['is using wrap to wrap a pair of skis.',
   'is ripping level tiles off.',
   "is holding a rubik's cube.",
   'starts pulling up roofing on a roof.'],
  'source_id': 'activitynet~v_-JhWjGDPHMY',
  'split': 'val',
  'split_type': 'indomain',
  'label': '3',
  'query': 'Roof shingle removal: A man is sitting on a roof. He',
  'choices': ['is using wrap to wrap a pair of skis.',
   'is ripping level tiles off.',
   "is holding a rubik's cube.",
   'starts pulling up roofing on a roof.'],
  'gold': 3},
 'target': 3,
 'arguments': [['Roof shingle removal: A man is sitting on a roof. He',
   ' is using wrap to wrap a pair of skis.'],
  ['Roof shingle removal: A man is sitting on a roof. He',
   ' is ripping level tiles off.'],
  ['Roof shingle removal: A man is sitting on a roof. He',
   " is holding

In [13]:
import json
from collections import Counter

data_paths = ["./datasets/split2_model7_cluster/gsm8k-train.json","./datasets/split2_model7_cluster/humaneval_train.json", "./datasets/split2_model7_cluster/arc_challenge_train.json", "./datasets/split2_model7_cluster/mmlu_train.json","./datasets/split2_model7_cluster/cmmlu_train.json",]
cluster_counters = {}
for data_path in data_paths:
    with open(data_path, "r") as f:
        dataset_file = json.load(f)
    for data_item in dataset_file:
        if data_item['cluster_id'] in cluster_counters:
            for key, value in data_item['scores'].items():
                if value > 0:
                    cluster_counters[data_item['cluster_id']][key] += 1
        else:
            cluster_counters[ data_item['cluster_id']] = Counter()

model_list = ['mistralai/Mistral-7B-v0.1',
                'meta-math/MetaMath-Mistral-7B',
                'itpossible/Chinese-Mistral-7B-v0.1',
                "HuggingFaceH4/zephyr-7b-beta",
                "cognitivecomputations/dolphin-2.6-mistral-7b",
                "meta-llama/Meta-Llama-3-8B",
                "cognitivecomputations/dolphin-2.9-llama3-8b"]
    
cluster_model_map = []
for i in range(5):
    suitable_model_name, _ = cluster_counters[i].most_common()[0]
    suitable_model_index = model_list.index(suitable_model_name)
    cluster_model_map.append(suitable_model_index)

cluster_model_map

[3, 6, 5, 5, 4]

In [18]:
import json
from collections import Counter
import os


output_path = "./datasets/lora_retriever"
os.makedirs(output_path, exist_ok=True)
data_paths = ["./datasets/split2_model7_cluster/gsm8k-train.json","./datasets/split2_model7_cluster/humaneval_train.json", "./datasets/split2_model7_cluster/arc_challenge_train.json", "./datasets/split2_model7_cluster/mmlu_train.json","./datasets/split2_model7_cluster/cmmlu_train.json",]
cluster_counters = {}

cluster_samples_dict = {}

for data_path in data_paths:
    with open(data_path, "r") as f:
        dataset_file = json.load(f)
    for data_item in dataset_file:
        if data_item['cluster_id'] in cluster_samples_dict:
            cluster_samples_dict[data_item['cluster_id']].append(data_item)
        else:
            cluster_samples_dict[data_item['cluster_id']] = [data_item]
        if data_item['cluster_id'] in cluster_counters:
            for key, value in data_item['scores'].items():
                if value > 0:
                    cluster_counters[data_item['cluster_id']][key] += 1
        else:
            cluster_counters[ data_item['cluster_id']] = Counter()


for cluster_id, cluster_sample_list in cluster_samples_dict.items():
    with open(os.path.join(output_path, f"cluster_{cluster_id}.json"), "w") as f:
        json.dump(cluster_sample_list, f)

model_list = ['mistralai/Mistral-7B-v0.1',
                'meta-math/MetaMath-Mistral-7B',
                'itpossible/Chinese-Mistral-7B-v0.1',
                "HuggingFaceH4/zephyr-7b-beta",
                "cognitivecomputations/dolphin-2.6-mistral-7b",
                "meta-llama/Meta-Llama-3-8B",
                "cognitivecomputations/dolphin-2.9-llama3-8b"]
    
cluster_model_map = []
for i in range(5):
    suitable_model_name, _ = cluster_counters[i].most_common()[0]
    suitable_model_index = model_list.index(suitable_model_name)
    cluster_model_map.append(suitable_model_index)

cluster_model_map

[3, 6, 5, 5, 4]

In [ ]:
model_list = [  ['mistralai','Mistral-7B-v0.1'],
                ['meta-math','MetaMath-Mistral-7B'],
                ['teknium','OpenHermes-2.5-Mistral-7B'],
                ['upaya07','Arithmo2-Mistral-7B'],
                # ['ajibawa-2023','Code-Mistral-7B'],
                ['WizardLM', 'WizardMath-7B-V1.1'],
                ['itpossible','Chinese-Mistral-7B-v0.1'],
                ["HuggingFaceH4","zephyr-7b-beta"],
                ["cognitivecomputations","dolphin-2.6-mistral-7b"],
                ["Open-Orca","Mistral-7B-OpenOrca"],
                ["HuggingFaceH4","mistral-7b-grok"],
                ["HuggingFaceH4","mistral-7b-sft-beta"],
                ["NousResearch","Hermes-2-Pro-Mistral-7B"],
                ["cognitivecomputations","dolphin-2.2.1-mistral-7b"],
                ["abhishekchohan","mistral-7B-forest-dpo"],
                ["wandb","mistral-7b-zephyr-sft"],
                ["Locutusque","Hercules-2.5-Mistral-7B"],
                ["nvidia","OpenMath-Mistral-7B-v0.1-hf"]]





In [36]:
# check nan

import pandas as pd 
for i in pd.read_json("./datasets/one_task/humaneval_train.json")["scores"]:
    for value in i.values():
        if value == None:
            print("nan")